In [9]:
import pandas as pd
df = pd.read_csv(r'polemik.csv')
df.head()

,Unnamed: 0,Komentar,Label
0,1,Lha kekerasan seksual itu kan mmg definisinya ...,positif
1,2,Jangan mendekati zina. Itu adalah aturan yang ...,negatif
2,3,The law is an art. Semakin rinci semakin mudah...,positif
3,4,TETAPKAN DULU APA YANG DIMAKSUD DENGAN KEKERAS...,netral
4,5,Lbih cock diterapkan dikemendikbud Dan DPR.,netral


In [13]:
import re
import nltk
import pandas as pd


def casefolding(review):
    review = review.lower()
    return review

def tokenize(review):
    token = nltk.word_tokenize(review)
    return token

def filtering(review):
    # Remove angka termasuk angka yang berada dalam string
    # Remove non ASCII chars
    review = re.sub(r'[^\x00-\x7f]', r'', review)
    review = re.sub(r'(\\u[0-9A-Fa-f]+)', r'', review)
    review = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", review)
    review = re.sub(r'\\u\w\w\w\w', '', review)
    # Remove link web
    review = re.sub(r'http\S+', '', review)
    # Remove @username
    review = re.sub('@[^\s]+', '', review)
    # Remove #tagger
    review = re.sub(r'#([^\s]+)', '', review)
    # Remove simbol, angka dan karakter aneh
    review = re.sub(r"[.,:;+!\-_<^/=?\"'\(\)\d\*]", " ", review)
    return review

def replaceThreeOrMore(review):
    # Pattern to look for three or more repetitions of any character, including newlines (contoh goool -> gol).
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1", review)

def convertToSlangword(review):
    kamus_slangword = eval(open("combined_slang_words.txt").read()) # Membuka dictionary slangword
    pattern = re.compile(r'\b( ' + '|'.join (kamus_slangword.keys())+r')\b') # Search pola kata (contoh kpn -> kapan)
    content = []
    for kata in review:
        filteredSlang = pattern.sub(lambda x: kamus_slangword[x.group()],kata) # Replace slangword berdasarkan pola review yg telah ditentukan
        content.append(filteredSlang.lower())
    review = content
    return review

def removeStopword(review):
    stopwords = open('combined_stop_words.txt', 'r').read().split()
    content = []
    filteredtext = [word for word in review.split() if word not in stopwords]
    content.append(" ".join(filteredtext))
    review = content
    return review

# stringData = casefolding(String)
# print (stringData)

data = df
# Keeping only the neccessary columns
datasets = [data]
   
for teks in datasets:
    label = teks['Label']
    teks = teks['Komentar'].apply(casefolding)
    teks = teks.apply(filtering)
    teks = teks.apply(replaceThreeOrMore)
    teks = teks.apply(tokenize)
    teks = teks.apply(convertToSlangword)
    teks = teks.apply(" ".join)
    teks = teks.apply(removeStopword)
    teks = teks.apply(" ".join)
    print(teks)

review_dict = {'Komentar': teks, 'Label' : label}
df = pd.DataFrame(review_dict, columns = ['Komentar', 'Label'])
print(df.info())
df.to_csv('data-bersih.csv', sep= ',' , encoding='utf-8')

0      kekerasan seksual kan memang definisinya meman...
1      mendekati zina aturan pasal dl sat demokrasi t...
2                    law is an art rinci mudah dibelokan
3                         tetapkan kekerasan seksualitas
4                     cocok diterapkan dikemendikbud dpr
                             ...                        
641    ganti menterinya sekolah pendidikan amburadul ...
642                                   hukum mati selesai
643                                  laki laki perempuan
644                                                germo
645    gawat darurat dikampus karenahukum jalan nadim...
Name: Komentar, Length: 646, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 646 entries, 0 to 645
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Komentar  646 non-null    object
 1   Label     646 non-null    object
dtypes: object(2)
memory usage: 10.2+ KB
None
